# Print with clay at CODA ExperienceLab

This notebook can be used as a basis to create shapes for clay 3D printing. 

It uses polar equations to define the shape to be printed. Actually a polar equation is the only thing needed to create a new shape.

The GenerativeShape module contains necessary functions that will generate the printing instructions layer by layer.

The easiest way to get started is to run the whole notebook (Menu Run > Run All Cells) then visualize the result. It will also generate the gcode file ready to print which can be viewed in a gcode viewer

Then modify one of the methods defined in the Shell class then assign it to the shell.polar_curve attribute

The various curves used to define the shape can be explored here
https://www.geogebra.org/calculator/cfqafypg

printing parameters are defined for a Vormvrij Lutum 5M printer

Visualisation and gcode generation rely on the FullControl package which can be found here https://github.com/FullControlXYZ/fullcontrol

In [ ]:
import numpy as np
import fullcontrol as fc
from math import pi, tau, sin, cos, exp
from generative_printing import GenerativeShape
import ipynbname

In [ ]:
# shape dimensions (mm)
vertical_scale = 140.0
horizontal_scale = 70.0

# 3D printing parameters
extrusion_width = 4.0
layer_height = 1.3
first_layer_height = 1.0
filled_bottom_layers = 0
printer_name='lutum' # generic / ultimaker2plus / prusa_i3 / ender_3 / cr_10 / bambulab_x1 / toolchanger_T0
file_name = ipynbname.name()


In [ ]:
class Shell(GenerativeShape):
    """
    The three methods below define each a different shape.
    The print_polar method will call the selected method once for each layer that needs to be printed.
    The methods return polar coordinates of the layer contour.
    
    The total number of layers needed to reach the desired height is calculated in print_polar and stored in self.layer_count
    layer_index is the current layer index that goes from 0 to self.layer_count-1
    
    h is the height of the current layer relative to the final height of the shape as defined in vertical_scale.
    So h takes values between 0 to 1, both included.
    This makes it easier to define polar equations that will create a shape then scale it to the desired size
    
    """
    def polar_curve_bumpy(self, layer_index, h):
        points_par_layer = 1000
        perimeter_coefficient = self.horizontal_scale
        t = np.linspace(0, tau, points_par_layer)
        p1 = 0.2 - 0.8 * (h+0.2) * (h+1.4) * (h-1.2)
        # rho = perimeter_coefficient * np.ones(points_par_layer)
        # rho = perimeter_coefficient * (1+0.05*np.sin(12*t)*np.sin(12*pi*h))
        rho = perimeter_coefficient * (1+0.05*np.sin(12*t)*np.sin(12*pi*h)) * p1
        theta = t
        return((rho,theta))  

    def polar_curve_twisted(self, layer_index, h):
        pattern_periods_per_turn = 5
        points_per_period = 200
        points_par_layer = pattern_periods_per_turn * points_per_period
        perimeter_coefficient = self.horizontal_scale
        t = np.linspace(0, tau, points_par_layer)
        p1 = 0.2 - 0.8 * (h+0.2) * (h+1.4) * (h-1.2)
        rho = perimeter_coefficient * (1+0.05*np.sin(12*t+6*pi*h)) * p1
        theta = t
        return((rho,theta))  
    
    def polar_curve_deep_sea_shell(self, layer_index, h):
        pattern_periods_per_turn = 5
        points_per_period = 200
        points_par_layer = pattern_periods_per_turn * points_per_period
        perimeter_coefficient = self.horizontal_scale
        
        t = np.linspace(0, tau, points_par_layer)

        p3 = 0.9 * (h - 1.4) * (h + 0.2) * (h - 1.8)
        
        # rho = perimeter_coefficient * np.ones(points_par_layer)
        # rho = perimeter_coefficient * (1+0.3*np.cos(pattern_periods_per_turn*t))
        rho = perimeter_coefficient * (1+0.3*np.cos(pattern_periods_per_turn*t)) * p3

        p2 = 0.28-0.3*(h+0.1)*(h+1)*(h-1)
        
        # theta = t
        # theta = t + 0.3 * np.sin(2 * pattern_periods_per_turn * t)
        theta = t + np.sin(2 * pattern_periods_per_turn * t) * p2
        
        return((rho,theta))

In [ ]:
# Create the shape
shell = Shell(
    vertical_scale=vertical_scale,
    horizontal_scale = horizontal_scale,
    layer_height=layer_height,
    first_layer_height=first_layer_height,
    extrusion_width = extrusion_width,
)

# Uncomment one of the following three lines to generate the different shapes
shell.polar_curve = shell.polar_curve_deep_sea_shell
# shell.polar_curve = shell.polar_curve_twisted
# shell.polar_curve = shell.polar_curve_deep_sea_shell

# Create the printing steps
shell.print_polar()

# Since polar equations are centered on the origine, the shape must be moved to the positive quadrant for printing 
steps = shell.move_to_positive_quadrant()

In [ ]:
# Preview the design

fc.transform(steps, 'plot', fc.PlotControls(style='line', zoom=0.7))

In [ ]:
# Generate gcode file
initial_settings = {
    "print_speed": 2400,
    "travel_speed": 6000,
    "extrusion_width": extrusion_width,
    "extrusion_height": layer_height,
}

gcode_controls = fc.GcodeControls(
    printer_name=printer_name,
    initialization_data=initial_settings,
    save_as=file_name,
    include_date=True)

gcode_string = fc.transform(steps,'gcode', gcode_controls)